# GRIK Test(Renewal)

## 1. Import

In [7]:
## 설치 안된 Dependency를 처리하기 위해
import sys
import subprocess

In [8]:
## 파이썬 기본 라이브러리
import os
from PIL import Image
from datetime import datetime
import shutil

## 라이브러리가 없으면 설치를 시도하고 이미 설치되어 있으면 import만 한다.
try:
	import cv2
except:
    # pip 모듈 업그레이드
    subprocess.check_call([sys.executable,'-m', 'pip', 'install', '--upgrade', 'pip'])
    # 에러 발생한 모듈 설치
    subprocess.check_call([sys.executable,'-m', 'pip', 'install', '--upgrade', 'opencv-python'])
    import cv2

try:
    from skimage.metrics import structural_similarity as ssim
except:
    # 에러 발생한 모듈 설치
    subprocess.check_call([sys.executable,'-m', 'pip', 'install', '--upgrade', 'scikit-image'])
    from skimage.metrics import structural_similarity as ssim

## 2. 비디오 인식

In [9]:
## 비디오 경로
video_Path = input("비디오 경로를 입력하세요: ")

## opencv 비디오 인식
video = cv2.VideoCapture(video_Path)

## 비디오가 없을 때
if not video.isOpened():
    print(f'Cannot open the video: {video_Path}')

In [10]:
## 비디오 정보
length = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = round(video.get(cv2.CAP_PROP_FPS))

## 디렉토리 생성
save_NAME = input("저장할 이름을 입력해주세요: ")
save_DIR = f'./result/{save_NAME}'
try:
    if not os.path.exists(save_DIR):
        os.makedirs(save_DIR)
except OSError:
    print(f'Error: Cannot Find SAVE_PATH: {save_DIR}')

## SSIM

In [11]:
## Define Frame Counter
frameCounter = 0

## Define SSIM pre-processor
def SSIMProcessor(first, second, thisTurn):
    ## Filtered Gray Scale
    grayA = cv2.cvtColor(first, cv2.COLOR_BGR2GRAY)
    grayB = cv2.cvtColor(second, cv2.COLOR_BGR2GRAY)

    ## Saving Score
    (score, diff) = ssim(grayA, grayB, full=True)
    diff = (diff * 225).astype("uint8")

    ## If Score is larger than 0.87, then delete Primary
    if thisTurn == 1:
        cv2.imwrite(f'{save_DIR}/frame{thisTurn - 1}.jpg', first)
        print(f'#{thisTurn - 1} Frame Saved!!')
    if score < 0.8:
        cv2.imwrite(f'{save_DIR}/frame{thisTurn - 1}.jpg', first)
        print(f'#{thisTurn - 1} Frame Saved!!')
        cv2.imwrite(f'{save_DIR}/frame{thisTurn}.jpg', second)
        print(f'#{thisTurn} Frame Saved!!')


## Video Pre-processing
while(video.isOpened()):
    ret, image = video.read()
    if (int(video.get(1)) % fps == 0):
        if (frameCounter == 0):
            Primary = image
        else :
            Secondary = image
            SSIMProcessor(Primary, Secondary, frameCounter)
            Primary = Secondary
        frameCounter += 1
    if int(video.get(1)) == length:
        cv2.imwrite(f'{save_DIR}/frame{frameCounter}.jpg', image)
        break

video.release()

## Remove Video
#os.remove(video_Path)
#print("Eliminate Video!!")

#0 Frame Saved!!
#60 Frame Saved!!
#61 Frame Saved!!
#265 Frame Saved!!
#266 Frame Saved!!
#745 Frame Saved!!
#746 Frame Saved!!
#1446 Frame Saved!!
#1447 Frame Saved!!
#1783 Frame Saved!!
#1784 Frame Saved!!
#3254 Frame Saved!!
#3255 Frame Saved!!
#3621 Frame Saved!!
#3622 Frame Saved!!
